Importación de librerias y seteo de variables constantes

In [ ]:
import pandas as pd
from extraccion import Extraccion
from bson.decimal128 import Decimal128
import matplotlib.pyplot as plt

URL = "mongodb://localhost:27017/"
DATABASE_NAME = "AirBnbDB"

Creación de instancia de la clase Extracción y lectura de colecciones para almacenamiento en dataframe

In [ ]:
extraccion = Extraccion(URL, DATABASE_NAME)
extraccion.conectar()

Lectura y análisis de listings

In [ ]:
listings:pd.DataFrame = extraccion.obtener_datos("listado")

print("Primeras filas de Listings:")
print(listings.head())

print(f"Número de filas: {listings.shape[0]} \n columnas en Listings: {listings.shape[1]}")

print("Tipos de datos en Listings:")
listings.info()

Lectura y análisis de reviews

In [ ]:
reviews:pd.DataFrame = extraccion.obtener_datos("reseñas")

print("\nPrimeras filas de Reviews:")
print(reviews.head())

print(f"Número de filas: {reviews.shape[0]} \n columnas en Listings: {reviews.shape[1]}")

print("Tipos de datos en Reviews:")
reviews.info()

Lectura y análisis de calendar

In [ ]:
calendar:pd.DataFrame = extraccion.obtener_datos("calendario")


print("\nPrimeras filas de Calendar:")
print(calendar.head())

print(f"Número de filas: {calendar.shape[0]} \n columnas en Listings: {calendar.shape[1]}")

print("Tipos de datos en Calendar:")
calendar.info()

Entendimiento general de los datos

                                Análisis de nulos para cada dataframe

Nulos en listings

In [ ]:
print("Valores nulos de listings:")
print(listings.isnull().sum())

print("\nAnálisis de valores nulos en Listings:")
valores_nulos_listings = listings.isnull().sum()
print(valores_nulos_listings)

plt.figure(figsize=(10, 6))
valores_nulos_listings[valores_nulos_listings > 0].plot(kind='bar', color='skyblue')
plt.title("Valores nulos por columna en Listings")
plt.xlabel("Columnas")
plt.ylabel("Cantidad de valores nulos")
plt.xticks(rotation=45)
plt.show()

Nulos en Reviews

In [ ]:
print("\nAnálisis de valores nulos en Reviews:")
valores_nulos_reviews = reviews.isnull().sum()
print(valores_nulos_reviews)

plt.figure(figsize=(10, 6))
valores_nulos_reviews[valores_nulos_reviews > 0].plot(kind='bar', color='lightgreen')
plt.title("Valores nulos por columna en Reviews")
plt.xlabel("Columnas")
plt.ylabel("Cantidad de valores nulos")
plt.xticks(rotation=45)
plt.show()

Nulos en Calendar

In [ ]:
print("\nAnálisis de valores nulos en Calendar:")
valores_nulos_calendar = calendar.isnull().sum()
print(valores_nulos_calendar)

plt.figure(figsize=(10, 6))
valores_nulos_calendar[valores_nulos_calendar > 0].plot(kind='bar', color='salmon')
plt.title("Valores nulos por columna en Calendar")
plt.xlabel("Columnas")
plt.ylabel("Cantidad de valores nulos")
plt.xticks(rotation=45)
plt.show()

Análisis duplicados listings

In [ ]:
listingsDuplicate = listings.copy()

#Se realizan las conversiones necesarias para evitar errores al buscar duplicados

for col in listingsDuplicate.columns:
    if listingsDuplicate[col].apply(lambda x: isinstance(x, list)).any():
        listingsDuplicate[col] = listingsDuplicate[col].apply(lambda x: str(x) if isinstance(x, list) else x)

for col in listingsDuplicate.columns:
    if listingsDuplicate[col].apply(lambda x: isinstance(x, Decimal128)).any():
        listingsDuplicate[col] = listingsDuplicate[col].apply(lambda x: float(x.to_decimal()) if isinstance(x, Decimal128) else x)

print(listingsDuplicate.duplicated().sum())

#Para esta colección no eliminamos los duplicados

Análisis duplicados reviews

In [ ]:
print("\nAnálisis de duplicados en Reviews:")
reviewsDuplicate = reviews.copy()

for col in reviewsDuplicate.columns:
    if reviewsDuplicate[col].apply(lambda x: isinstance(x, list)).any():
        reviewsDuplicate[col] = reviewsDuplicate[col].apply(lambda x: str(x) if isinstance(x, list) else x)
    if reviewsDuplicate[col].apply(lambda x: isinstance(x, Decimal128)).any():
        reviewsDuplicate[col] = reviewsDuplicate[col].apply(lambda x: float(x.to_decimal()) if isinstance(x, Decimal128) else x)

duplicados_reviews = reviewsDuplicate.duplicated().sum()
print(f"Duplicados en Reviews: {duplicados_reviews}")

if duplicados_reviews > 0:
    reviewsDuplicate.drop_duplicates(inplace=True)
    print("Duplicados eliminados en Reviews.")

#En esta eliminamos duplicados para no afectar reseñas para los diferentes sitios

Análisis de duplicados para calendar

In [ ]:
print("\nAnálisis de duplicados en Calendar:")
calendarDuplicate = calendar.copy()

for col in calendarDuplicate.columns:
    if calendarDuplicate[col].apply(lambda x: isinstance(x, list)).any():
        calendarDuplicate[col] = calendarDuplicate[col].apply(lambda x: str(x) if isinstance(x, list) else x)
    if calendarDuplicate[col].apply(lambda x: isinstance(x, Decimal128)).any():
        calendarDuplicate[col] = calendarDuplicate[col].apply(lambda x: float(x.to_decimal()) if isinstance(x, Decimal128) else x)

duplicados_calendar = calendarDuplicate.duplicated().sum()
print(f"Duplicados en Calendar: {duplicados_calendar}")

if duplicados_calendar > 0:
    calendarDuplicate.drop_duplicates(inplace=True)
    print("Duplicados eliminados en Calendar.")

#En esta si eliminamos registros duplicados porque estarían alterando análisis 

Validación de valores atípicos para campos price, minimum_nights, availability_365

In [ ]:

for columna in ["price", "minimum_nights", "availability_365"]:
    if columna in listings.columns:
        listings[columna] = pd.to_numeric(listings[columna], errors='coerce')
        if listings[columna].isnull().sum() > 0:
            print(f"Advertencia: La columna '{columna}' contiene valores no numéricos. Se convertirán a NaN.")
            print(listings[listings[columna].isnull()])

